In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from imutils import paths
import imutils
import numpy as np
import cv2
import os
import dlib

In [2]:
## Load model
model=load_model(r'face_mask_detector.h5')


In [3]:
prototxtPath = 'deploy.prototxt.txt'
weightsPath= 'res10_300x300_ssd_iter_140000.caffemodel'
faceNet=cv2.dnn.readNet(weightsPath,prototxtPath)

In [12]:
def detect_and_predict_mask(frame,faceNet,model):
    (h,w)=frame.shape[:2]
    blob=cv2.dnn.blobFromImage(frame,1.0,(300,300),(104.0,177.0,123.0))
    
    faceNet.setInput(blob)
    detections=faceNet.forward()
    
    faces=[]
    locs=[]
    preds=[]
    
    for i in range(0,detections.shape[2]):
        confidence=detections[0,0,i,2]
    
        if confidence>0.5:
            box=detections[0,0,i,3:7]*np.array([w,h,w,h])
            (startX,startY,endX,endY)=box.astype('int')
        
            (startX,startY)=(max(0,startX-30),max(0,startY-30))
            (endX,endY)=(min(w-1,endX+30), min(h-1,endY+30))
        
            face=frame[startY:endY, startX:endX]
            face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
            face=cv2.resize(face,(96,96))
            face=img_to_array(face)
            face=preprocess_input(face)
        
            faces.append(face)
            locs.append((startX,startY,endX,endY))
        
    if len(faces)>0:
        faces=np.array(faces,dtype='float32')
        preds=model.predict(faces,batch_size=6)
        
    return (locs,preds)

In [43]:
def test_on_video(video, model):
    success, frame = video.read()
    while success:
        success, frame = video.read()
        if(not success):
            break
        frame=imutils.resize(frame,width=800)
        (locs,preds)=detect_and_predict_mask(frame,faceNet,model)
        
        for (box,pred) in zip(locs,preds):
            (startX,startY,endX,endY)=box
            (mask,withoutMask)=pred

            label='Mask' if mask>0.5 else 'No Mask'
            color=(0,255,0) if label=='Mask' else (0,0,255)
            label="{}: {:.2f}%".format(label,max(mask,withoutMask)*100)
            cv2.putText(frame,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
            cv2.rectangle(frame,(startX,startY),(endX,endY),color,2)
        cv2.imshow("Frame",frame)
        key=cv2.waitKey(1) & 0xFF

        if key==ord('q'):
            break

    cv2.destroyAllWindows()
    vs.stop()

In [52]:
video1='face_mask.mp4'
cap = cv2.VideoCapture(video1)
test_on_video(cap, model);

1/1 [==============================] - 0s 13ms/step


1/1 [==============================] - 0s 13ms/step


NameError: name 'vs' is not defined